### Import Required Libraries and Set Up Environment Variables

In [5]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv('environment.env')
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [7]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2014-05-01"

# Build URL for CME
url = base_url + CME + '?startDate=' + startDate + '&endDate=' + endDate + '&api_key=' + NASA_API_KEY
print (url)

https://api.nasa.gov/DONKI/CME?startDate=2013-05-01&endDate=2014-05-01&api_key=ECLDEpEzmZ5MFaU5IBR1hRCNVBmdcBFjhgujhClv


In [9]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(url)

In [11]:
# Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [13]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json, indent=4))

[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "speed": 860.0,
        

In [15]:
# Convert cme_json to a Pandas DataFrame 
cme_df = pd.DataFrame(cme_json)
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = cme_df[['activityID', 'startTime', 'linkedEvents']]
cme_df

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
...,...,...,...
566,2014-04-29T20:48:00-CME-001,2014-04-29T20:48Z,None
567,2014-04-29T21:28:00-CME-001,2014-04-29T21:28Z,None
568,2014-04-30T14:36:00-CME-001,2014-04-30T14:36Z,None
569,2014-05-01T08:24:00-CME-001,2014-05-01T08:24Z,None


In [17]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df = cme_df.dropna(subset = 'linkedEvents')
cme_df.reset_index(drop=True, inplace=True)
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
4,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [19]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for index in cme_df.index:
    activityID = cme_df.loc[index,'activityID']
    startTime = cme_df.loc[index,'startTime']
    linkedEvents = cme_df.loc[index,'linkedEvents']
    # Iterate over each dictionary in the list
    for event in linkedEvents:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({'activityID':activityID,         
                              'startTime': startTime,
                              'linkedEvents': event})
# Create a new DataFrame from the expanded rows
expanded_df = pd.DataFrame(expanded_rows)
expanded_df

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}
...,...,...,...
179,2014-04-09T00:00:00-CME-001,2014-04-09T00:00Z,{'activityID': '2014-04-12T02:30:00-IPS-001'}
180,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-18T12:31:00-FLR-001'}
181,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-18T15:25:00-SEP-001'}
182,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-20T10:22:00-IPS-001'}


In [21]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

        # Log the error or print it for debugging

def extract_activityID_from_dict(event): 
    try:
        return event['activityID']
        print (linkedEvent)
    except KeyError:
        print('No linked event found')
print(event)

{'activityID': '2014-04-25T00:17:00-FLR-001'}


,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
179,2014-04-09T00:00:00-CME-001,2014-04-09T00:00Z,{'activityID': '2014-04-12T02:30:00-IPS-001'},2014-04-12T02:30:00-IPS-001
180,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-18T12:31:00-FLR-001'},2014-04-18T12:31:00-FLR-001
181,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-18T15:25:00-SEP-001'},2014-04-18T15:25:00-SEP-001
182,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-20T10:22:00-IPS-001'},2014-04-20T10:22:00-IPS-001


In [27]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_df['GST_ActivityID'] = expanded_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))
expanded_df

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
179,2014-04-09T00:00:00-CME-001,2014-04-09T00:00Z,{'activityID': '2014-04-12T02:30:00-IPS-001'},2014-04-12T02:30:00-IPS-001
180,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-18T12:31:00-FLR-001'},2014-04-18T12:31:00-FLR-001
181,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-18T15:25:00-SEP-001'},2014-04-18T15:25:00-SEP-001
182,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-20T10:22:00-IPS-001'},2014-04-20T10:22:00-IPS-001


In [29]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_df = expanded_df.dropna(subset = 'GST_ActivityID')
expanded_df.reset_index(drop=True, inplace=True)
expanded_df

,activityID,startTime,linkedEvents,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001
...,...,...,...,...
179,2014-04-09T00:00:00-CME-001,2014-04-09T00:00Z,{'activityID': '2014-04-12T02:30:00-IPS-001'},2014-04-12T02:30:00-IPS-001
180,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-18T12:31:00-FLR-001'},2014-04-18T12:31:00-FLR-001
181,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-18T15:25:00-SEP-001'},2014-04-18T15:25:00-SEP-001
182,2014-04-18T13:09:00-CME-001,2014-04-18T13:09Z,{'activityID': '2014-04-20T10:22:00-IPS-001'},2014-04-20T10:22:00-IPS-001


In [33]:
# print out the datatype of each column in this DataFrame:
expanded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      184 non-null    object
 1   startTime       184 non-null    object
 2   linkedEvents    184 non-null    object
 3   GST_ActivityID  184 non-null    object
dtypes: object(4)
memory usage: 5.9+ KB


In [35]:
# Convert the 'GST_ActivityID' column to string format 
expanded_df['GST_ActivityID'] = expanded_df['GST_ActivityID'].astype('str')
# Convert startTime to datetime format  
expanded_df['startTime'] = pd.to_datetime(expanded_df['startTime'])
# Rename startTime to startTime_CME and activityID to cmeID
expanded_df = expanded_df.rename(columns={'startTime': 'startTime_CME', 'activityID': 'cmeID'})
# Drop linkedEvents
expanded_df = expanded_df.drop('linkedEvents', axis=1)
# Verify that all steps were executed correctly
expanded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           184 non-null    object             
 1   startTime_CME   184 non-null    datetime64[ns, UTC]
 2   GST_ActivityID  184 non-null    object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 4.4+ KB


In [37]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
filtered_cme_df = expanded_df[expanded_df['GST_ActivityID'].str.contains('GST', na=False)]
filtered_cme_df.reset_index(drop=True, inplace=True)
filtered_cme_df

,cmeID,startTime_CME,GST_ActivityID
0,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001
5,2014-02-25T01:25:00-CME-001,2014-02-25 01:25:00+00:00,2014-02-27T18:00:00-GST-001


### GST Data

In [39]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2014-05-01"

# Build URL for GST
gst_url = base_url+GST+"?startDate="+startDate+"&endDate="+endDate+"&api_key="+NASA_API_KEY
print(gst_url)

https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2014-05-01&api_key=ECLDEpEzmZ5MFaU5IBR1hRCNVBmdcBFjhgujhClv


In [41]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(gst_url)

In [43]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()
# Preview the first result in JSON format
gst_json
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json, indent=4))

[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    },
    {
        "gstID": "2013-06-07T03:00:00-GST-001",
        "startTime": "2013-06-07T03:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-07T03:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/330/-1",
        "linkedEvents": [
            {
                "activityID": "2013-06-02T20:24:

In [45]:
# Convert gst_json to a Pandas DataFrame  
gst_df = pd.DataFrame(gst_json)
# Keep only the columns: activityID, startTime, linkedEvents
gst_df = gst_df[['gstID','startTime', 'linkedEvents']]
gst_df

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...
6,2014-02-20T03:00:00-GST-001,2014-02-20T03:00Z,[{'activityID': '2014-02-18T01:25:00-CME-001'}...
7,2014-02-27T18:00:00-GST-001,2014-02-27T18:00Z,[{'activityID': '2014-02-25T01:25:00-CME-001'}...


In [59]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df = gst_df.dropna(subset = 'linkedEvents')
gst_df.reset_index(drop=True, inplace=True)
gst_df

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
3,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
4,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...
5,2014-02-20T03:00:00-GST-001,2014-02-20T03:00Z,[{'activityID': '2014-02-18T01:25:00-CME-001'}...
6,2014-02-27T18:00:00-GST-001,2014-02-27T18:00Z,[{'activityID': '2014-02-25T01:25:00-CME-001'}...


In [103]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df_exploded = gst_df.explode('linkedEvents')
gst_df_exploded.reset_index(drop=True, inplace=True)
gst_df_exploded

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}
5,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,{'activityID': '2013-12-04T23:12:00-CME-001'}
6,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,{'activityID': '2013-12-07T21:30:00-HSS-001'}
7,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,{'activityID': '2014-02-16T14:15:00-CME-001'}
8,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,{'activityID': '2014-02-19T03:10:00-IPS-001'}
9,2014-02-20T03:00:00-GST-001,2014-02-20T03:00Z,{'activityID': '2014-02-18T01:25:00-CME-001'}


In [113]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df_exploded.loc[:,'CME_ActivityID'] = gst_df_exploded['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df_exploded = gst_df_exploded.dropna(subset = 'CME_ActivityID')
gst_df_exploded.reset_index(drop=True, inplace=True)
gst_df_exploded

,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001
5,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,{'activityID': '2013-12-04T23:12:00-CME-001'},2013-12-04T23:12:00-CME-001
6,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,{'activityID': '2013-12-07T21:30:00-HSS-001'},2013-12-07T21:30:00-HSS-001
7,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,{'activityID': '2014-02-16T14:15:00-CME-001'},2014-02-16T14:15:00-CME-001
8,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,{'activityID': '2014-02-19T03:10:00-IPS-001'},2014-02-19T03:10:00-IPS-001
9,2014-02-20T03:00:00-GST-001,2014-02-20T03:00Z,{'activityID': '2014-02-18T01:25:00-CME-001'},2014-02-18T01:25:00-CME-001


In [115]:
# Convert the 'CME_ActivityID' column to string format 
gst_df_exploded['CME_ActivityID'] = gst_df_exploded['CME_ActivityID'].astype(str)
# Convert the 'gstID' column to string format 
gst_df_exploded['gstID'] = gst_df_exploded['gstID'].astype(str)
# Convert startTime to datetime format  
gst_df_exploded['startTime'] = pd.to_datetime(gst_df_exploded['startTime'])
# Rename startTime to startTime_GST 
gst_df_exploded = gst_df_exploded.rename(columns = {'startTime': 'startTime_GST'})
# Drop linkedEvents
gst_df_exploded = gst_df_exploded.drop('linkedEvents', axis = 1)
# Verify that all steps were executed correctly
gst_df_exploded

,gstID,startTime_GST,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01 01:00:00+00:00,2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T02:47:00-MPC-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
6,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-07T21:30:00-HSS-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
8,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-19T03:10:00-IPS-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


In [127]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
filtered_gst_df = gst_df_exploded[gst_df_exploded['CME_ActivityID'].str.contains('CME', na=False)]
filtered_gst_df.reset_index(drop=True, inplace=True)
filtered_gst_df

,gstID,startTime_GST,CME_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001
5,2014-02-27T18:00:00-GST-001,2014-02-27 18:00:00+00:00,2014-02-25T01:25:00-CME-001


### Merge both datatsets

In [131]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(filtered_gst_df,filtered_cme_df,left_on=['gstID', 'CME_ActivityID'], right_on=['GST_ActivityID', 'cmeID'], how='inner')
merged_df

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001
5,2014-02-27T18:00:00-GST-001,2014-02-27 18:00:00+00:00,2014-02-25T01:25:00-CME-001,2014-02-25T01:25:00-CME-001,2014-02-25 01:25:00+00:00,2014-02-27T18:00:00-GST-001


In [133]:
# Verify that the new DataFrame has the same number of rows as cme and gst
# print (f"number of rows in CME DF = {}
if filtered_cme_df.shape[0] == filtered_gst_df.shape[0]:
    print (f"number of rows in filtered_cme_df are: {(filtered_cme_df).shape[0]}")
    print (f"number of rows in filtered_gst_df are: {(filtered_gst_df).shape[0]}")
    print (f"number of rows in merged_df are: {(merged_df).shape[0]}")
else:
    print("Keyerror")

number of rows in filtered_cme_df are: 6
number of rows in filtered_gst_df are: 6
number of rows in merged_df are: 6


### Computing the time it takes for a CME to cause a GST

In [139]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df.loc[:,'timeDiff'] = merged_df.loc[:,'startTime_GST']-merged_df.loc[:,'startTime_CME']
merged_df

,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID,timeDiff
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001,4 days 06:36:00
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001,2 days 04:20:00
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001,3 days 00:48:00
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001,2 days 12:45:00
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001,2 days 01:35:00
5,2014-02-27T18:00:00-GST-001,2014-02-27 18:00:00+00:00,2014-02-25T01:25:00-CME-001,2014-02-25T01:25:00-CME-001,2014-02-25 01:25:00+00:00,2014-02-27T18:00:00-GST-001,2 days 16:35:00


In [141]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
merged_df.describe()

,timeDiff
count,6
mean,2 days 19:06:30
std,0 days 19:18:34.196247960
min,2 days 01:35:00
25%,2 days 06:26:15
50%,2 days 14:40:00
75%,2 days 22:44:45
max,4 days 06:36:00


### Exporting data in csv format

In [143]:
# Export data to CSV without the index
merged_df.to_csv("output/collected_data.csv", encoding="utf-8", index=False)